In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, broadcast
from datetime import timedelta

In [0]:
spark.conf.set("spark.sql.session.timeZone", "UTC")

In [0]:
alerts_path = "abfss://datos@mastertfm002sta.dfs.core.windows.net/gold/alerts"

# spark.sql(f"""
#     DELETE FROM delta.`{alerts_path}`
# """)

news_path = "abfss://datos@mastertfm002sta.dfs.core.windows.net/gold/config/news"


NEWS_LOOKBACK_MINUTES = 1440


In [0]:
##### BLOQUE 1 #####

gold_path = "abfss://datos@mastertfm002sta.dfs.core.windows.net/gold/activos"
estrategias_csv_path = "abfss://datos@mastertfm002sta.dfs.core.windows.net/gold/config/estrategias_optimas.csv"
alerts_path = "abfss://datos@mastertfm002sta.dfs.core.windows.net/gold/alerts"
checkpoint_path = alerts_path + "/_checkpoint_inference"


In [0]:
##### BLOQUE 2 #####


spark = (
    SparkSession.builder
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)



In [0]:
##### BLOQUE 3 #####

estrategias_df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(estrategias_csv_path)
)

# # Opción A, filtrar por pasa_filtro si existe
# if "pasa_filtro" in estrategias_df.columns:
#     estrategias_apto = estrategias_df.filter(col("pasa_filtro") == True)
# else:
#     # Opción B, filtrar por clasificacion
# estrategias_apto = estrategias_df.filter(col("clasificacion").isin(["APTO FUERTE 🚀"]))

estrategias_apto = estrategias_df.filter(col("clasificacion").isin(["APTO FUERTE 🚀"])).select(
    "symbol",
    "tipo_estrategia",   # 👈 NUEVO
    "best_x",
    "best_k",
    "best_n",
    "best_j"
)

display(estrategias_apto)


symbol,tipo_estrategia,best_x,best_k,best_n,best_j
SPCE,Rebote,47,2,19,0.002
SOXX,Inercia,51,2,19,2.0E-4
ARKK,Rebote,29,2,19,2.0E-4
PLTR,Rebote,37,3,16,4.0E-4
LCID,Inercia,53,6,16,0.0
COIN,Rebote,36,3,19,0.002
VUG,Rebote,44,2,14,4.0E-4
SPY,Rebote,40,3,18,0.0
QQQ,Inercia,35,2,12,0.001


In [0]:
##### BLOQUE 4 #####

gold_replay = (
    spark.read
    .format("delta")
    .load(gold_path)
    .filter(col("timestamp") >= "2026-02-01")
    .filter(col("timestamp") < "2026-02-16")
    .orderBy("timestamp")
)

# gold tiene: timestamp, symbol, asset_class, asset_name, coste_opera_h, close
stream_joined = (
    gold_replay
    .select("timestamp", "symbol", "asset_name", "close")
    .join(
        broadcast(estrategias_apto),
        on="symbol",
        how="inner"
    )
)

stream_joined.select("symbol", "best_x").distinct().show(50, False)


+------+------+
|symbol|best_x|
+------+------+
|SOXX  |51    |
|VUG   |44    |
|ARKK  |29    |
|LCID  |53    |
|QQQ   |35    |
|SPY   |40    |
|SPCE  |47    |
|PLTR  |37    |
|COIN  |36    |
+------+------+



In [0]:
##### BLOQUE 5 #####

from collections import deque, defaultdict
import pandas as pd
import numpy as np
import uuid

# Buffers y control de spam en DRIVER
BUFFERS = defaultdict(lambda: deque(maxlen=10_000))
LAST_ALERT_BIN = {}

# Convertir estrategias_apto (Spark DF) a dict en driver
# Asumimos pocos activos (decenas)
STRATEGIES = {
    r["symbol"]: {
        "tipo_estrategia": r["tipo_estrategia"],  # 👈 NUEVO
        "best_x": int(r["best_x"]),
        "best_k": int(r["best_k"]),
        "best_n": int(r["best_n"]),
        "best_j": float(r["best_j"])
    }
    for r in estrategias_apto.collect()
}


print(f"✅ Estrategias cargadas: {len(STRATEGIES)} activos")


# ============================
# ESTADO DE POSICIONES EN DRIVER
# ============================

# Una posición por activo
# guardamos: entry_price, entry_time, peak_price (para trailing), last_bin (anti spam ya lo tienes aparte)
POSITIONS = {}

# Si quieres que el trailing sea un porcentaje fijo, define esto:
# TRAIL_PCT_DEFAULT = 0.002  # 0.2%

# Si quieres usar best_j como trailing, lo haremos dentro del bloque 9 usando best_j



✅ Estrategias cargadas: 9 activos


In [0]:
##### BLOQUE 6 #####

# ============================
# WARM-UP HISTÓRICO DESDE GOLD
# ============================

from datetime import timedelta
from pyspark.sql.functions import col
from pyspark.sql import functions as F
import pandas as pd

print("🔥 Iniciando warm-up histórico desde GOLD")

# ✅ Corte absoluto: desde aquí NO se usa nada hacia atrás
CUTOFF_TS = pd.Timestamp("2026-02-01 00:00:00")

# 1️⃣ Calcular minutos necesarios (peor estrategia)
estrategias_pdf = estrategias_apto.toPandas()
estrategias_pdf["min_needed"] = (
    (estrategias_pdf["best_k"] + estrategias_pdf["best_n"] + 5)
    * estrategias_pdf["best_x"]
)
max_minutes_needed = int(estrategias_pdf["min_needed"].max())
print(f"⏱️ Ventana teórica necesaria: {max_minutes_needed} minutos (pero SIN usar datos < {CUTOFF_TS})")

# 2️⃣ Timestamp de inicio del replay (primer dato >= cutoff)
replay_start_epoch = (
    spark.read
    .format("delta")
    .load(gold_path)
    .filter(col("timestamp") >= F.lit(str(CUTOFF_TS)))
    .select(F.min(col("timestamp").cast("long")).alias("ts"))
    .collect()[0]["ts"]
)

replay_start_ts = pd.to_datetime(replay_start_epoch, unit="s", utc=True).tz_convert("UTC").tz_localize(None)
print(f"🕒 Inicio del replay: {replay_start_ts}")

# 3️⃣ Leer GOLD histórico SOLO desde cutoff (sin periodo hacia atrás)
symbols_estrategia = estrategias_pdf["symbol"].tolist()

gold_hist = (
    spark.read
    .format("delta")
    .load(gold_path)
    .filter(col("symbol").isin(symbols_estrategia))
    .filter(col("timestamp") >= F.lit(str(CUTOFF_TS)))
    .filter(col("timestamp") <= F.lit(str(replay_start_ts)))
)

# 4️⃣ Pasar a pandas y cargar buffers (puede quedar corto al principio, es intencional)
pdf_hist = (
    gold_hist
    .select("timestamp", "symbol", "close")
    .orderBy("symbol", "timestamp")
    .toPandas()
)

for symbol, g in pdf_hist.groupby("symbol"):
    row = estrategias_pdf[estrategias_pdf["symbol"] == symbol].iloc[0]
    min_raw_needed = int((row["best_k"] + row["best_n"] + 5) * row["best_x"])

    BUFFERS[symbol] = deque(maxlen=min_raw_needed * 2)

    # ✅ Clampeo explícito por seguridad
    g = g[g["timestamp"] >= CUTOFF_TS]

    for _, r in g.iterrows():
        BUFFERS[symbol].append((r["timestamp"], float(r["close"])))

    print(f"✅ Warm-up {symbol}: {len(BUFFERS[symbol])} puntos cargados desde {CUTOFF_TS}")


🔥 Iniciando warm-up histórico desde GOLD
⏱️ Ventana teórica necesaria: 1431 minutos (pero SIN usar datos < 2026-02-01 00:00:00)
🕒 Inicio del replay: 2026-02-01 00:00:00
✅ Warm-up ARKK: 1 puntos cargados desde 2026-02-01 00:00:00
✅ Warm-up COIN: 1 puntos cargados desde 2026-02-01 00:00:00
✅ Warm-up LCID: 1 puntos cargados desde 2026-02-01 00:00:00
✅ Warm-up PLTR: 1 puntos cargados desde 2026-02-01 00:00:00
✅ Warm-up QQQ: 1 puntos cargados desde 2026-02-01 00:00:00
✅ Warm-up SOXX: 1 puntos cargados desde 2026-02-01 00:00:00
✅ Warm-up SPCE: 1 puntos cargados desde 2026-02-01 00:00:00
✅ Warm-up SPY: 1 puntos cargados desde 2026-02-01 00:00:00
✅ Warm-up VUG: 1 puntos cargados desde 2026-02-01 00:00:00


In [0]:
##### BLOQUE 7 #####

from pyspark.sql.types import (
    StructType, StructField,
    TimestampType, StringType,
    IntegerType, DoubleType
)

alert_schema = StructType([
    StructField("trade_id", StringType(), False),
    StructField("tipo_estrategia", StringType(), False),  # 👈 NUEVO
    StructField("timestamp_alert", TimestampType(), False),

    StructField("asset_name", StringType(), False),
    StructField("symbol", StringType(), False),

    StructField("best_x", IntegerType(), False),
    StructField("best_k", IntegerType(), False),
    StructField("best_n", IntegerType(), False),
    StructField("best_j", DoubleType(), False),

    StructField("signal", StringType(), False),
    StructField("price", DoubleType(), False),
    StructField("bin_start", TimestampType(), False),

    StructField("entry_price", DoubleType(), True),
    StructField("exit_reason", StringType(), True),
    StructField("pnl_pct", DoubleType(), True),
    StructField("hold_minutes", IntegerType(), True),

    StructField("news_count", IntegerType(), True),
    StructField("news_main_title", StringType(), True),
    StructField("news_main_link", StringType(), True),
    StructField("news_main_published_utc", TimestampType(), True),
    StructField("news_time_diff_minutes", IntegerType(), True),
    StructField("news_main_sentiment_label", StringType(), True),
    StructField("news_main_sentiment_score", DoubleType(), True),

])



In [0]:
##### BLOQUE 8 #####

import numpy as np
import pandas as pd

def _last_k_returns(prices: np.ndarray) -> np.ndarray:
    """Retornos simples por barra: prices[t]/prices[t-1]-1"""
    if len(prices) < 2:
        return np.array([], dtype=float)
    prev = prices[:-1]
    nxt = prices[1:]
    # evitar divisiones raras
    prev = np.where(prev <= 1e-12, np.nan, prev)
    r = (nxt / prev) - 1.0
    return r

def compute_entry_signal(prices: np.ndarray, k: int, j: float, tipo_estrategia: str):
    """
    Señal de ENTRADA según tu definición.

    - Inercia:
        BUY  si las últimas k barras tienen retorno >= j
        SELL si las últimas k barras tienen retorno <= -j
    - Rebote (contrario):
        BUY  si las últimas k barras tienen retorno <= -j
        SELL si las últimas k barras tienen retorno >= j

    return: "BUY" | "SELL" | None
    """
    tipo = (tipo_estrategia or "").strip().lower()

    r = _last_k_returns(prices)
    if len(r) < k:
        return None

    last = r[-k:]
    if np.any(np.isnan(last)):
        return None

    if tipo == "inercia":
        if np.all(last > j):
            return "BUY"
        if np.all(last < -j):
            return "SELL"
        return None

    if tipo == "rebote":
        if np.all(last < -j):
            return "BUY"
        if np.all(last > j):
            return "SELL"
        return None

    # si viene algo raro, no operamos
    return None


In [0]:
##### BLOQUE 9 #####

import traceback
from pyspark.sql.functions import col
from pyspark.sql import Row
import uuid
import pandas as pd
from collections import deque

# ⬇️ imports mínimos necesarios para el join con noticias (no cambia nada más)
from pyspark.sql.functions import expr, row_number, unix_timestamp, abs as sql_abs
from pyspark.sql import Window

GOLD_BUFFERS = {}

# ✅ Estado por activo en DRIVER
# position: "LONG" | "SHORT"
# bars_held: número de barras (x minutos) completadas desde la entrada
POSITIONS = {}

# ✅ Para no procesar ni alertar 2 veces el mismo bin
LAST_PROCESSED_BIN_END = {}
LAST_ALERT_BIN_END = {}

# ✅ Corte absoluto
CUTOFF_TS = pd.Timestamp("2026-02-01 00:00:00")

def _pnl_pct(side: str, entry: float, last: float) -> float:
    if entry <= 0 or last <= 0:
        return 0.0
    if side == "LONG":
        return (last / entry) - 1.0
    if side == "SHORT":
        return (entry / last) - 1.0
    return 0.0

def process_inference_batch(batch_df, batch_id: int):
    try:
        sdf = (
            batch_df
            .select("timestamp_epoch", "symbol", "asset_name", "close")
            .filter(col("symbol").isin(list(STRATEGIES.keys())))
        )

        pdf = sdf.toPandas()
        nrows = len(pdf)
        print(f"[INFERENCE] batch_id={batch_id} rows={nrows}")

        # =========================
        # CARGAR NOTICIAS (STATIC)
        # =========================
        news_df = (
            spark.read
            .format("delta")
            .load(news_path)
            .select(
                "asset",
                "title",
                "link",
                col("published_utc").alias("news_published_utc"),
                col("sentiment").alias("sentiment_label"),   # ✅ AQUÍ,
                col("sentiment_score")
            )
        )

        if nrows == 0:
            print(f"[INFERENCE] batch_id={batch_id} vacío")
            return

        pdf["timestamp"] = (
            pd.to_datetime(pdf["timestamp_epoch"], unit="s", utc=True)
            .dt.tz_convert("UTC")
            .dt.tz_localize(None)
        )

        # ✅ Corte absoluto: ignorar cualquier fila < 2026-01-01
        pdf = pdf[pdf["timestamp"] >= CUTOFF_TS]

        if len(pdf) == 0:
            print(f"[INFERENCE] batch_id={batch_id} todo fuera de rango (< {CUTOFF_TS})")
            return

        pdf = pdf.sort_values(["symbol", "timestamp"])

        t_min = pdf["timestamp"].min()
        t_max = pdf["timestamp"].max()
        print(f"[INFERENCE] batch_id={batch_id} data_range={t_min} → {t_max}")

        alerts_out = []

        for symbol, g in pdf.groupby("symbol"):

            params = STRATEGIES.get(symbol)
            if not params:
                continue

            best_x = int(params["best_x"])
            best_k = int(params["best_k"])
            best_n = int(params["best_n"])
            best_j = float(params["best_j"])
            tipo_estrategia = (params.get("tipo_estrategia") or "").lower().strip()

            asset_name = g["asset_name"].iloc[0]

            # tamaño de buffer recomendado
            min_raw_needed = (best_k + best_n + 5) * best_x

            if symbol not in BUFFERS:
                BUFFERS[symbol] = deque(maxlen=min_raw_needed * 2)
            if BUFFERS[symbol].maxlen < min_raw_needed * 2:
                BUFFERS[symbol] = deque(BUFFERS[symbol], maxlen=min_raw_needed * 2)

            # cargar ticks del batch al buffer
            for _, row in g.iterrows():
                ts = row["timestamp"]
                if ts < CUTOFF_TS:
                    continue
                BUFFERS[symbol].append((ts, float(row["close"])))

            # series para resample
            buf = list(BUFFERS[symbol])
            if len(buf) < 2:
                continue

            # =========================
            # USAR GOLD DIRECTAMENTE (CON BUFFER HISTÓRICO)
            # =========================

            if symbol not in GOLD_BUFFERS:
                GOLD_BUFFERS[symbol] = deque(maxlen=5000)

            for _, row in g.iterrows():
                ts = row["timestamp"]
                if ts < CUTOFF_TS:
                    continue
                GOLD_BUFFERS[symbol].append((ts, float(row["close"])))

            gold_df_sym = (
                pd.DataFrame(GOLD_BUFFERS[symbol], columns=["timestamp", "close"])
                .drop_duplicates("timestamp")
                .sort_values("timestamp")
            )

            gold_minute_df = (
                pd.DataFrame(GOLD_BUFFERS[symbol], columns=["timestamp", "close"])
                .drop_duplicates("timestamp")
                .sort_values("timestamp")
            )

            now_ts_data = g["timestamp"].max()

            gold_df_sym = gold_df_sym[gold_df_sym["timestamp"] <= now_ts_data]

            mins_from_cutoff = ((gold_df_sym["timestamp"] - CUTOFF_TS).dt.total_seconds() // 60).astype(int)
            gold_df_sym = gold_df_sym[mins_from_cutoff % best_x == 0]

            if len(gold_df_sym) < best_k + 1:
                continue

            last_bin_end_ts = gold_df_sym["timestamp"].iloc[-1]
            last_bin_end = last_bin_end_ts.to_pydatetime()

            last_bin_start = last_bin_end_ts - pd.Timedelta(minutes=best_x)

            last_price = float(gold_df_sym["close"].iloc[-1])

            prices = gold_df_sym["close"].astype(float).values

            if last_price <= 0:
                continue

            prev_bin = LAST_PROCESSED_BIN_END.get(symbol)
            if prev_bin is not None and prev_bin == last_bin_end:
                continue
            LAST_PROCESSED_BIN_END[symbol] = last_bin_end

            prices = gold_df_sym["close"].astype(float).values

            # =========================
            # 1) Señal de ENTRADA (si no hay posición)
            # =========================
            pos = POSITIONS.get(symbol)

            if pos is None:

                prices_np = np.array(prices, dtype=float)

                # gold_df_sym ya son cierres cerrados y alineados
                prices_closed = prices_np

                if len(prices_closed) < best_k + 1:
                    continue

                entry_signal = compute_entry_signal(
                    prices=prices_closed,
                    k=best_k,
                    j=best_j,
                    tipo_estrategia=tipo_estrategia
                )

                if entry_signal in ("BUY", "SELL"):
                    if LAST_ALERT_BIN_END.get(symbol) == last_bin_end:
                        continue
                    LAST_ALERT_BIN_END[symbol] = last_bin_end

                    bin_close_price = float(prices_closed[-1])

                    trade_id = str(uuid.uuid4())
                    side = "LONG" if entry_signal == "BUY" else "SHORT"

                    POSITIONS[symbol] = {
                        "trade_id": trade_id,
                        "side": side,
                        "entry_price": bin_close_price,
                        "entry_time": last_bin_end,
                        "bars_held": 0,
                        "x_minutes": best_x,
                        "n_bars": best_n,
                        "last_bin_end": last_bin_end,
                    }

                    alerts_out.append(Row(
                        trade_id=trade_id,
                        tipo_estrategia=tipo_estrategia,
                        timestamp_alert=last_bin_end,
                        asset_name=asset_name,
                        symbol=symbol,
                        best_x=best_x,
                        best_k=best_k,
                        best_n=best_n,
                        best_j=best_j,
                        signal=entry_signal,
                        price=bin_close_price,
                        bin_start=last_bin_start.to_pydatetime(),
                        entry_price=bin_close_price,
                        exit_reason=None,
                        pnl_pct=None,
                        hold_minutes=None,
                        news_count=None,
                        news_main_title=None,
                        news_link=None,
                        news_published_utc=None,
                        news_time_diff_minutes=None,
                        sentiment_label=None,
                        sentiment_score=None
                    ))

                continue

            # =========================
            # 2) Gestión de POSICIÓN (salida por tiempo n barras)
            # =========================
            if "last_bin_end" not in pos:
                pos["last_bin_end"] = last_bin_end
                pos["bars_held"] = 0
            elif pos["last_bin_end"] != last_bin_end:
                pos["bars_held"] += 1
                pos["last_bin_end"] = last_bin_end

            bars_held = int(pos["bars_held"])
            n_bars = int(pos["n_bars"])
            side = pos["side"]
            trade_id = pos["trade_id"]

            if bars_held >= n_bars:
                exit_signal = "SELL" if side == "LONG" else "BUY"

                bin_close_price = float(prices[-1])

                pnl = _pnl_pct(side, float(pos["entry_price"]), bin_close_price)
                hold_minutes = bars_held * int(pos["x_minutes"])

                if LAST_ALERT_BIN_END.get(symbol) != last_bin_end:
                    LAST_ALERT_BIN_END[symbol] = last_bin_end

                    alerts_out.append(Row(
                        trade_id=trade_id,
                        tipo_estrategia=tipo_estrategia,
                        timestamp_alert=last_bin_end,
                        asset_name=asset_name,
                        symbol=symbol,
                        best_x=best_x,
                        best_k=best_k,
                        best_n=best_n,
                        best_j=best_j,
                        signal=exit_signal,
                        price=bin_close_price,
                        bin_start=last_bin_start.to_pydatetime(),
                        entry_price=float(pos["entry_price"]),
                        exit_reason="TIME_BASED_EXIT",
                        pnl_pct=round(pnl * 100, 2),
                        hold_minutes=int(hold_minutes),
                        news_count=None,
                        news_main_title=None,
                        news_link=None,
                        news_published_utc=None,
                        news_time_diff_minutes=None,
                        sentiment_label=None,
                        sentiment_score=None

                    ))

                del POSITIONS[symbol]

        if not alerts_out:
            print(f"[INFERENCE] batch_id={batch_id} sin alertas")
            return

        alerts_sdf = spark.createDataFrame(alerts_out, schema=alert_schema)

        # =========================
        # ENRIQUECER ALERTAS CON NOTICIAS
        # =========================

        alerts_with_news = (
            alerts_sdf.alias("a")
            .join(
                news_df.alias("n"),
                on=(
                    (col("a.symbol") == col("n.asset")) &
                    (col("n.news_published_utc") <= col("a.timestamp_alert")) &
                    (col("n.news_published_utc") >= expr(
                        f"a.timestamp_alert - INTERVAL {NEWS_LOOKBACK_MINUTES} MINUTES"
                    ))
                ),
                how="left"
            )
        )

        w_news = (
            Window
            .partitionBy("a.trade_id")
            .orderBy(
                col("n.news_published_utc").desc(),
                sql_abs(col("n.sentiment_score")).desc()
            )
        )

        alerts_ranked = alerts_with_news.withColumn("news_rank", row_number().over(w_news))

        alerts_main_news = (
            alerts_ranked
            .filter(col("news_rank") == 1)
            .withColumn(
                "news_time_diff_minutes",
                (
                    unix_timestamp(col("a.timestamp_alert")) -
                    unix_timestamp(col("n.news_published_utc"))
                ) / 60
            )
        )

        news_counts = (
            alerts_ranked
            .groupBy("a.trade_id")
            .count()
            .withColumnRenamed("count", "news_count_near")
        )


        alerts_enriched = (
            alerts_main_news
            .join(news_counts, on="trade_id", how="left")
            .select(
                col("a.trade_id"),
                col("a.tipo_estrategia"),
                col("a.timestamp_alert"),
                col("a.asset_name"),
                col("a.symbol"),
                col("a.best_x"),
                col("a.best_k"),
                col("a.best_n"),
                col("a.best_j"),
                col("a.signal"),
                col("a.price"),
                col("a.bin_start"),
                col("a.entry_price"),
                col("a.exit_reason"),
                col("a.pnl_pct"),
                col("a.hold_minutes"),

                # 🔹 noticias (ya sin duplicados)
                col("news_count_near").alias("news_count"),
                col("n.title").alias("news_main_title"),
                col("n.link").alias("news_main_link"),
                col("n.news_published_utc").alias("news_main_published_utc"),
                col("news_time_diff_minutes").cast("int"),
                col("n.sentiment_label").alias("news_main_sentiment_label"),
                col("n.sentiment_score").alias("news_main_sentiment_score"),
            )
        )


        (
            alerts_enriched.write
            .format("delta")
            .mode("append")
            .option("mergeSchema", "true")
            .save(alerts_path)
        )

        print(f"[INFERENCE] batch_id={batch_id} ALERTAS={alerts_out.__len__()}")

        alerts_enriched.select(
            "timestamp_alert",
            "symbol",
            "signal",
            "exit_reason",
            "pnl_pct",
            "hold_minutes",
            "news_count",
            "news_main_published_utc",
            "news_main_title",
            "news_main_link",
            "news_main_sentiment_label",
            "news_main_sentiment_score",
            "trade_id"
        ).orderBy("timestamp_alert", "symbol").show(truncate=False)

    except Exception:
        print("🔥 ERROR REAL EN INFERENCE FOREACH BATCH 🔥")
        traceback.print_exc()
        raise



In [0]:
##### BLOQUE 10 #####

from pyspark.sql import functions as F

replay_batches = (
    gold_replay
    .withColumn("timestamp_epoch", F.col("timestamp").cast("long"))  # segundos epoch UTC
    .groupBy("timestamp_epoch")
    .agg(
        F.collect_list(
            F.struct("timestamp_epoch", "asset_name", "symbol", "close")
        ).alias("rows")
    )
    .orderBy("timestamp_epoch")
    .collect()
)

print(f"🔥 Iniciando replay histórico: {len(replay_batches)} minutos")



🔥 Iniciando replay histórico: 21600 minutos


In [0]:
##### BLOQUE 11 #####

batch_id = 0
for b in replay_batches:
    rows = b["rows"]
    if not rows:
        print(f"[REPLAY] batch_id={batch_id} vacío")
        batch_id += 1
        continue

    from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType

    batch_schema = StructType([
        StructField("timestamp_epoch", LongType(), False),
        StructField("asset_name", StringType(), True),
        StructField("symbol", StringType(), True),
        StructField("close", DoubleType(), True),
    ])

    sdf = spark.createDataFrame(rows, schema=batch_schema)

    process_inference_batch(sdf, batch_id)
    batch_id += 1



[INFERENCE] batch_id=0 rows=9
[INFERENCE] batch_id=0 data_range=2026-02-01 00:00:00 → 2026-02-01 00:00:00
[INFERENCE] batch_id=0 sin alertas
[INFERENCE] batch_id=1 rows=9
[INFERENCE] batch_id=1 data_range=2026-02-01 00:01:00 → 2026-02-01 00:01:00
[INFERENCE] batch_id=1 sin alertas
[INFERENCE] batch_id=2 rows=9
[INFERENCE] batch_id=2 data_range=2026-02-01 00:02:00 → 2026-02-01 00:02:00
[INFERENCE] batch_id=2 sin alertas
[INFERENCE] batch_id=3 rows=9
[INFERENCE] batch_id=3 data_range=2026-02-01 00:03:00 → 2026-02-01 00:03:00
[INFERENCE] batch_id=3 sin alertas
[INFERENCE] batch_id=4 rows=9
[INFERENCE] batch_id=4 data_range=2026-02-01 00:04:00 → 2026-02-01 00:04:00
[INFERENCE] batch_id=4 sin alertas
[INFERENCE] batch_id=5 rows=9
[INFERENCE] batch_id=5 data_range=2026-02-01 00:05:00 → 2026-02-01 00:05:00
[INFERENCE] batch_id=5 sin alertas
[INFERENCE] batch_id=6 rows=9
[INFERENCE] batch_id=6 data_range=2026-02-01 00:06:00 → 2026-02-01 00:06:00
[INFERENCE] batch_id=6 sin alertas
[INFERENCE] b

[INFERENCE] batch_id=15128 rows=9
[INFERENCE] batch_id=15128 data_range=2026-02-11 12:08:00 → 2026-02-11 12:08:00
[INFERENCE] batch_id=15128 sin alertas
[INFERENCE] batch_id=15129 rows=9
[INFERENCE] batch_id=15129 data_range=2026-02-11 12:09:00 → 2026-02-11 12:09:00
[INFERENCE] batch_id=15129 sin alertas
[INFERENCE] batch_id=15130 rows=9
[INFERENCE] batch_id=15130 data_range=2026-02-11 12:10:00 → 2026-02-11 12:10:00
[INFERENCE] batch_id=15130 sin alertas
[INFERENCE] batch_id=15131 rows=9
[INFERENCE] batch_id=15131 data_range=2026-02-11 12:11:00 → 2026-02-11 12:11:00
[INFERENCE] batch_id=15131 sin alertas
[INFERENCE] batch_id=15132 rows=9
[INFERENCE] batch_id=15132 data_range=2026-02-11 12:12:00 → 2026-02-11 12:12:00
[INFERENCE] batch_id=15132 sin alertas
[INFERENCE] batch_id=15133 rows=9
[INFERENCE] batch_id=15133 data_range=2026-02-11 12:13:00 → 2026-02-11 12:13:00
[INFERENCE] batch_id=15133 sin alertas
[INFERENCE] batch_id=15134 rows=9
[INFERENCE] batch_id=15134 data_range=2026-02-11